[GSE217483](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE217483)

在这里，我们分析了感染严重急性呼吸系统综合征冠状病毒2型的恒河猴的肺、肝、肾和大脑皮层的约77000个单核转录组。


| tissue | status | batch | gender | macacaID | macacaID_count |
| :--: | :--: | :--: | :--: | :--: | :--: |
| cortex | healthy | 0 | male | Ctrl1 | 1996 |
| cortex | healthy | 1 | male | Ctrl2 | 8349 |
| cortex | infected | 0 | female | Case1 | 5827 |
| cortex | infected | 0 | male | Case2 | 7434 |
| kidney | healthy | 0 | male | Ctrl1 | 5307 |
| kidney | infected | 0 | female | Case1 | 4733 |
| kidney | infected | 0 | male | Case2 | 3510 |
| liver | healthy | 0 | male | Ctrl1 | 4152 |
| liver | infected | 0 | female | Case1 | 7510 |
| liver | infected | 0 | male | Case2 | 6070 |
| lung | healthy | 0 | male | Ctrl1 | 5360 |
| lung | infected | 0 | female | Case1 | 9035 |
| lung | infected | 0 | male | Case2 | 7746 |

In [1]:
import sys
from pathlib import Path
p_root = Path('~/link/res_publish').expanduser()
None if str(p_root) in sys.path else sys.path.append(str(p_root))


In [2]:
from func import *

/public/workspace/licanchengup/apps/miniconda3/envs/publish/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-05-05 01:08:26.055220: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.



-------------------------help-------------------------
> parameter
    p_root	[name] res_publish
        p_run, p_plot, p_res, p_cache, p_pdf
    p_df_varmap
    map_sp_reverse
    rng
> run
    run_cross_species_models
    h5ad_to_mtx
    load_adata
    get_path_varmap
    find_path_from_para
    load_normalized_adata

> res
    get_test_result_df
    get_res_obs
    get_adata_umap
    show_umap

> plot
    get_color_map
    show_color_map
    show_color
    plot_umap
    savefig



In [3]:
adata = sc.read_h5ad('GSE217483_Macaca_global_integration.h5ad')
display(type(adata.X))
adata.X = csr_matrix(adata.X)
display(type(adata.X), adata)

scipy.sparse._csr.csr_matrix

scipy.sparse._csr.csr_matrix

AnnData object with n_obs × n_vars = 77029 × 13131
    obs: 'CC.difference', 'G2M_score', 'S_score', 'Unnamed..0', 'b025', 'batch', 'cell_type', 'doublet_score', 'fine_cell_type', 'gender', 'leiden', 'macacaID', 'n_counts', 'n_genes', 'nuclusID', 'organ', 'percent_mito', 'percent_ribo', 'phase', 'predicted_doublets', 'sampleID', 'status', 'tissue', 'broad_cell_type'
    var: 'gene_ids-0', 'feature_types', 'n_cells-0', 'gene_ids-1', 'n_cells-1'

In [4]:
temp = adata.X[:100, :100]
display(np.ravel(temp[temp > 0])[:10])
del temp

array([1., 3., 1., 1., 1., 1., 1., 1., 1., 3.], dtype=float32)

In [5]:
print(*adata.obs.columns, sep='\n')

CC.difference
G2M_score
S_score
Unnamed..0
b025
batch
cell_type
doublet_score
fine_cell_type
gender
leiden
macacaID
n_counts
n_genes
nuclusID
organ
percent_mito
percent_ribo
phase
predicted_doublets
sampleID
status
tissue
broad_cell_type


In [6]:
display(adata.obs.head(2))
# for i in 'sampleID,status,macacaID,batch,gender,organ'.split(','):
#     print(i.ljust(50,'-'),end='\n\n')
#     display(adata.obs[i].value_counts())
adata.obs = adata.obs.loc[:,
                          'sampleID,fine_cell_type,status,macacaID,batch,gender,organ'.split(',')]
adata.obs = adata.obs.rename(columns={'organ': 'tissue','fine_cell_type':'pre_fine_cell_type'})
adata.obs['macacaID'] = adata.obs['macacaID'].str.replace(
    'Ctrl', 'healthy_').str.replace('Case', 'infected_')
adata.obs['_batch'] = adata.obs['macacaID']
# new
adata.obs['cell_type'] = adata.obs['pre_fine_cell_type'].map(
    lambda k:{'Macrophage':'macrophage'}.setdefault(k,'')
)
adata.obs = adata.obs.loc[:,
                          '_batch,sampleID,cell_type,pre_fine_cell_type,status,macacaID,batch,gender,tissue'.split(',')]
display(adata.obs.head(2), adata.obs['pre_fine_cell_type'].value_counts())

,CC.difference,G2M_score,S_score,Unnamed..0,b025,batch,cell_type,doublet_score,fine_cell_type,gender,...,nuclusID,organ,percent_mito,percent_ribo,phase,predicted_doublets,sampleID,status,tissue,broad_cell_type
index,,,,,,,,,,,,,,,,,,,,,
AAACCCAAGCAAGCCA-1-0,-0.003720,-0.070341,-0.074062,1.0,True,0,AT2,0.149518,AT2,male,...,20200908.0,lung,0.000536,0.005365,G1,False,Macaca_mulatta_healthy_lung_20200911,healthy,0.0,Aveolar Epi Cells
AAACCCAAGTCGAAGC-1-0,0.092016,-0.136617,-0.044601,2.0,True,0,AT2,0.084946,AT2,male,...,20200908.0,lung,0.000000,0.005011,G1,False,Macaca_mulatta_healthy_lung_20200911,healthy,0.0,Aveolar Epi Cells


,_batch,sampleID,cell_type,pre_fine_cell_type,status,macacaID,batch,gender,tissue
index,,,,,,,,,
AAACCCAAGCAAGCCA-1-0,healthy_1,Macaca_mulatta_healthy_lung_20200911,,AT2,healthy,healthy_1,0,male,lung
AAACCCAAGTCGAAGC-1-0,healthy_1,Macaca_mulatta_healthy_lung_20200911,,AT2,healthy,healthy_1,0,male,lung


pre_fine_cell_type
Endothelial            6665
AT1                    6402
Zone1 Hepa             6028
Zone3 Hepa             5775
ExN_L2/3               5754
AT2                    5642
Fibroblast             2999
ExN_L5/6               2763
Macrophage             2562
ExN_L4                 2360
Proximal Tubule1       2124
Oligo                  1908
Neu_NRGN               1761
Astro                  1533
Proximal Tubule2       1528
InN_SST                1513
CD-PC                  1362
Proximal Tubule3       1227
InN_VIP                1174
InN_PVALB              1151
LOH3                    994
Interstitial Cells      938
Stellate cell1          892
OPC                     860
Connecting Tubule       839
Microglia               727
DCT                     721
T cell                  718
CD-IC type B            615
Zone2 Hepa              557
T Cell                  522
InN_RELN                473
CD-IC type A            470
Pericyte                463
Mesothelial             451
I

# lung

In [7]:
adata_l = adata[adata.obs['tissue'] == 'lung', :]
display(
    adata_l,
    adata_l.obs.head(2),
    adata_l.obs['tissue'].value_counts(),
    adata_l.obs['_batch'].value_counts(),
    adata_l.obs['pre_fine_cell_type'].value_counts())

View of AnnData object with n_obs × n_vars = 22141 × 13131
    obs: '_batch', 'sampleID', 'cell_type', 'pre_fine_cell_type', 'status', 'macacaID', 'batch', 'gender', 'tissue'
    var: 'gene_ids-0', 'feature_types', 'n_cells-0', 'gene_ids-1', 'n_cells-1'

,_batch,sampleID,cell_type,pre_fine_cell_type,status,macacaID,batch,gender,tissue
index,,,,,,,,,
AAACCCAAGCAAGCCA-1-0,healthy_1,Macaca_mulatta_healthy_lung_20200911,,AT2,healthy,healthy_1,0,male,lung
AAACCCAAGTCGAAGC-1-0,healthy_1,Macaca_mulatta_healthy_lung_20200911,,AT2,healthy,healthy_1,0,male,lung


tissue
lung    22141
Name: count, dtype: int64

_batch
infected_1    9035
infected_2    7746
healthy_1     5360
Name: count, dtype: int64

pre_fine_cell_type
AT1               6402
AT2               5642
Endothelial       3375
Fibroblast        2999
Macrophage        1630
Pericyte           463
Mesothelial        451
T Cell             329
Smooth_muscle      227
Lymphatic_Endo     177
Monocyte           144
Ciliated           125
Mast Cell          107
Clara               70
Name: count, dtype: int64

In [8]:
# new
adata_l_marc = adata_l[adata_l.obs['cell_type'] == 'macrophage']
display(adata_l_marc,adata_l_marc.obs.head(2),adata_l_marc.obs['cell_type'].value_counts())

View of AnnData object with n_obs × n_vars = 1630 × 13131
    obs: '_batch', 'sampleID', 'cell_type', 'pre_fine_cell_type', 'status', 'macacaID', 'batch', 'gender', 'tissue'
    var: 'gene_ids-0', 'feature_types', 'n_cells-0', 'gene_ids-1', 'n_cells-1'

,_batch,sampleID,cell_type,pre_fine_cell_type,status,macacaID,batch,gender,tissue
index,,,,,,,,,
AAACCCAGTGGTCTTA-1-0,healthy_1,Macaca_mulatta_healthy_lung_20200911,macrophage,Macrophage,healthy,healthy_1,0,male,lung
AAAGTGAGTCCTGTCT-1-0,healthy_1,Macaca_mulatta_healthy_lung_20200911,macrophage,Macrophage,healthy,healthy_1,0,male,lung


cell_type
macrophage    1630
Name: count, dtype: int64

In [9]:
adata_l_marc.obs['macr_cell_type'] =adata_l_marc.obs['cell_type']
adata_l_marc.obs = adata_l_marc.obs.loc[:,
                          '_batch,sampleID,cell_type,macr_cell_type,pre_fine_cell_type,status,macacaID,batch,gender,tissue'.split(',')]
display(adata_l_marc,adata_l_marc.obs.head(2))

/tmp/ipykernel_88336/42725173.py:1: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_l_marc.obs['macr_cell_type'] =adata_l_marc.obs['cell_type']


AnnData object with n_obs × n_vars = 1630 × 13131
    obs: '_batch', 'sampleID', 'cell_type', 'macr_cell_type', 'pre_fine_cell_type', 'status', 'macacaID', 'batch', 'gender', 'tissue'
    var: 'gene_ids-0', 'feature_types', 'n_cells-0', 'gene_ids-1', 'n_cells-1'

,_batch,sampleID,cell_type,macr_cell_type,pre_fine_cell_type,status,macacaID,batch,gender,tissue
index,,,,,,,,,,
AAACCCAGTGGTCTTA-1-0,healthy_1,Macaca_mulatta_healthy_lung_20200911,macrophage,macrophage,Macrophage,healthy,healthy_1,0,male,lung
AAAGTGAGTCCTGTCT-1-0,healthy_1,Macaca_mulatta_healthy_lung_20200911,macrophage,macrophage,Macrophage,healthy,healthy_1,0,male,lung


In [10]:
# display(adata.obs['tissue'].value_counts())
# for tissue in adata.obs['tissue'].unique():
#     h5ad_to_mtx(adata[adata.obs['tissue'] == tissue, :],
#                 'COVID_macaca_GSE217483_{}'.format(tissue))
# print("\n[finish]\n".center(100, "-"))
h5ad_to_mtx(adata_l_marc,
p_cache.joinpath('disease','COVID_ma_macrophage_GSE217483')
           )

frist 10 adata.X nonzero elements:
 [[8 1 1 1 3 1 1 1 1 1]]
[out] /public/workspace/licanchengup/link/res_publish/run/cache/disease/COVID_ma_macrophage_GSE217483


In [11]:
display(adata_l_marc.obs.head(2),adata_l_marc.obs.shape,
    adata_l_marc.obs['macacaID'].value_counts())

,_batch,sampleID,cell_type,macr_cell_type,pre_fine_cell_type,status,macacaID,batch,gender,tissue
index,,,,,,,,,,
AAACCCAGTGGTCTTA-1-0,healthy_1,Macaca_mulatta_healthy_lung_20200911,macrophage,macrophage,Macrophage,healthy,healthy_1,0,male,lung
AAAGTGAGTCCTGTCT-1-0,healthy_1,Macaca_mulatta_healthy_lung_20200911,macrophage,macrophage,Macrophage,healthy,healthy_1,0,male,lung


(1630, 10)

macacaID
infected_2    628
infected_1    625
healthy_1     377
Name: count, dtype: int64

# 与Mtb ma mac 合并

In [12]:
adata_l_marc.obs['note'] = adata_l_marc.obs['sampleID']
adata_l_marc.obs['status'] = adata_l_marc.obs['status'].map({
    'infected':'COVID19','healthy':'healthy'})
adata_l_marc.obs['treatment'] = adata_l_marc.obs['status']
adata_l_marc.obs = adata_l_marc.obs.loc[:,
    'status,_batch,treatment,cell_type,macr_cell_type,note'.split(',')]
display(adata_l_marc.obs.head(2),adata_l_marc.obs.shape)

,status,_batch,treatment,cell_type,macr_cell_type,note
index,,,,,,
AAACCCAGTGGTCTTA-1-0,healthy,healthy_1,healthy,macrophage,macrophage,Macaca_mulatta_healthy_lung_20200911
AAAGTGAGTCCTGTCT-1-0,healthy,healthy_1,healthy,macrophage,macrophage,Macaca_mulatta_healthy_lung_20200911


(1630, 6)

In [13]:
adata_ma_mtb_mac = load_adata(p_cache.joinpath('disease','Mtb_ma_macrophage_delCD8_SCP642'))

In [14]:
adata_ma_mtb_mac.obs['status'] = 'Mtb'
adata_ma_mtb_mac.obs['note'] = '_batch=treatment;tb_infection_dose;batch'
adata_ma_mtb_mac.obs = adata_ma_mtb_mac.obs.loc[:,
    'status,_batch,treatment,cell_type,macr_cell_type,note'.split(',')]
display(adata_ma_mtb_mac.obs.head(2),adata_ma_mtb_mac.obs.shape)

,status,_batch,treatment,cell_type,macr_cell_type,note
Array2_28918_CAACGACACATC,Mtb,CD8a_19_0,CD8a,macrophage,macrophage,_batch=treatment;tb_infection_dose;batch
Array2_28918_CAGTATGAGATC,Mtb,CD8a_19_0,CD8a,macrophage,macrophage,_batch=treatment;tb_infection_dose;batch


(18217, 6)

In [15]:
adata_merge_mac = sc.concat(
    {
        'Mtb':adata_ma_mtb_mac,
        'COVID19':adata_l_marc
    },index_unique=';'
)
display(adata_merge_mac.obs.head(2),adata_merge_mac.obs.shape)

,status,_batch,treatment,cell_type,macr_cell_type,note
Array2_28918_CAACGACACATC;Mtb,Mtb,CD8a_19_0,CD8a,macrophage,macrophage,_batch=treatment;tb_infection_dose;batch
Array2_28918_CAGTATGAGATC;Mtb,Mtb,CD8a_19_0,CD8a,macrophage,macrophage,_batch=treatment;tb_infection_dose;batch


(19847, 6)

In [16]:
h5ad_to_mtx(adata_merge_mac,p_cache.joinpath('disease','Mtb_COVID_merge_ma_macrophage'))

frist 10 adata.X nonzero elements:
 [[3 2 2 7 2 1 4 1 3 4]]
[out] /public/workspace/licanchengup/link/res_publish/run/cache/disease/Mtb_COVID_merge_ma_macrophage


In [17]:
print("\n[finish]\n".center(100,"-"))

---------------------------------------------
[finish]
---------------------------------------------
